In [1]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/merge_ssd'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.leaf_size_in_pages',
       'spec.index.search', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk', 'iostat.bytes_read',
       'iostat.bytes_wrtn'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.key_type,spec.load_sstable_in_mem,spec.name,spec.num_threads,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn
0,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,24971741285,24.971741,781256,4096,3200021488,399999998,781256,...,uint64,False,learned_merge_btree2561run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input1,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,6264496000,3687984000
1,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8214785197,8.214785,781255,4096,3200017088,219031081,74344,...,uint64,False,learned_merge_flatpgm25610run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3428224000,840044000
2,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8424467228,8.424467,781255,4096,3200017088,219031081,74344,...,uint64,False,learned_merge_btree25610run_1,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3436252000,867616000
3,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,7101072165,7.101072,781255,4096,3200016736,200199897,784,...,uint64,False,learned_merge_btree2561000run_1,8,sponge/merge_ssd/fb_threads=8/inputs/input1000,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3141980000,603352000
4,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8355656122,8.355656,781255,4096,3200017088,219031081,74344,...,uint64,False,standard_merge10run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/standard...,8,True,3428260000,897104000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,17388020872,17.388021,781250,4096,3199999984,219032474,74346,...,uint64,False,learned_merge_btree25610run_0,1,sponge/merge_ssd/fb_threads=1/inputs/input10,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,3436328000,3138264000
176,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,58861384880,58.861385,781250,4096,3199999984,399999998,781250,...,uint64,False,learned_merge_btree2561run_1,1,sponge/merge_ssd/fb_threads=1/inputs/input1,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,6264204000,3674140000
177,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,11868669099,11.868669,781250,4096,3199999984,201989891,7774,...,uint64,False,standard_merge100run_0,1,sponge/merge_ssd/fb_threads=1/inputs/input100,sponge/merge_ssd/fb_threads=1/outputs/standard...,8,True,3162072000,680348000
178,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,35775961271,35.775961,781250,4096,3199999984,399999998,781250,...,uint64,False,learned_merge_flatpgm2561run_1,1,sponge/merge_ssd/fb_threads=1/inputs/input1,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,6257216000,3674700000


In [3]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9
df['checksum'] = df['result.checksum']
df['threads'] = df['spec.num_threads']

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")
print(df['algo'].unique())


['learned_merge_btree256' 'learned_merge_flatpgm256' 'standard_merge']


In [6]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/merge_ssd/'+dataset+'.txt')



algos = ['standard_merge', 'learned_merge_flatpgm256',  'learned_merge_btree256']
print_for_dataset("fb", algos)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   ratio |   standard_merge |   learned_merge_flatpgm256 |   learned_merge_btree256 |
|--------:|-----------------:|---------------------------:|-------------------------:|
|       1 |          28.4662 |                   35.776   |                 58.6703  |
|      10 |          19.9407 |                   15.1024  |                 16.3015  |
|     100 |          18.2115 |                    7.9794  |                  7.81747 |
|    1000 |          16.234  |                    7.63789 |                  7.88183 |


In [20]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='books'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='osm'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))



algo,hash_join,inlj_btree256,inlj_sampledflatpgm256,lsj_btree256,lsj_sampledflatpgm256,sort_join
ratio,,,,,,
1,80.931304,49.388665,44.882598,55.784585,32.612449,31.299311
10,55.292828,8.003186,8.061300,8.363397,7.774270,8.724903
100,45.841991,6.726388,6.664628,6.787620,6.632520,7.177175
1000,21.057477,22.401654,18.125491,22.804902,18.035658,7.082692


algo,lsj_sampledflatpgm256
ratio,
1,32.612449
10,7.774270
100,6.632520
1000,18.035658


algo
ratio


algo
ratio


algo
ratio


algo
ratio


In [21]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))


algo
ratio


algo
ratio


algo
ratio


In [8]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('all_indexes/'+dataset+'.csv')



algos = ['lsj_sampledflatpgm256', 'lsj_btree256',  'sort_join', 'hash_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)



## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"

In [34]:
algos = ['lsj_sampledflatpgm256', 'lsj_flatpgm256', 'lsj_radixspline256', 'lsj_rmi' , 'lsj_btree256', 'sort_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)


## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_rmi |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|----------:|---------------:|------------:|
|       1 |               23.2294   |        22.4319   |            25.6702   | 20.3287   |       42.3009  |    11.7042  |
|      10 |                3.68178  |         3.45073  |             4.30738  |  3.76312  |        5.83421 |     4.45029 |
|     100 |                0.98362  |         1.01192  |             1.46279  |  1.25237  |        1.57196 |     3.12833 |
|    1000 |                0.385151 |         0.353513 |             0.374326 |  0.275802 |        0.31908 |     2.86744 |
## wiki
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|---------------:|------------:|
|       1 |               

In [29]:
algos = ['lsj_radixspline256', 'lsj_radixspline1024', 'lsj_radixspline4096']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)

## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"